## Rapport TME1 - Arbres de décision, sélection de modèles

Membres du binôme :
- KRISNI Almehdi
- ARICHANDRA Santhos

Dans ce TME, on aborde les aspects essentiels des arbres de décision. Les arbres sont des modèles de classification hiérarchique, en d'autres termes, pour des exemples de la forme **x** = *(x1, x2, ..., xd)*, on associe à chaque noeud de l'arbre un test sur une des dimensions de *xi* de la forme *xi* <= s avec s une valeur réelle. Le test indique le noeud fils devant être sélectionné. La feuille atteinte après l'ensemble des tests est la classe prédite pour un exemple.

### **L'essentiel sur les arbres de décision**

On cherche à implémenter les fonctions utiles au calcul du partitionnement optimal, soit les fonctions de calcul d'entropie. Toutes les méthodes sont situées dans le fichier *entropie.py* du répertoire TME1.

In [11]:
# Import du fichier entropie.py
from methodesTME import *

# Import utilitaires
import random
import numpy as np
import pickle

On code la méthode *entropie* permettant à partir d'un vecteur (une liste ou un vecteur numpy), de calculer l'entropie de ce vecteur. Une entropie importante représente un ensemble de données hétérogène tandis qu'une entropie nulle représente un ensemble homogéne, soit composé d'une seule et unique valeur.

On réalise quelques tests avec la méthode crée sur des vecteurs différents.

In [12]:
# Tests d'entropie
vT1 = [random.randint(0, 10) for _ in range(10)]
vT2 = [random.randint(0, 10) for _ in range(10)]
vT3 = [1 for _ in range(10)]

print("Vecteur :", vT1, "\tEntropie :", entropie(vT1))
print("Vecteur :", vT2, "\tEntropie :", entropie(vT2))
print("Vecteur :", vT3, "\tEntropie :", entropie(vT3))

Vecteur : [8, 6, 4, 0, 3, 2, 3, 3, 4, 6] 	Entropie : 1.6957425341696346
Vecteur : [8, 10, 6, 1, 0, 10, 10, 5, 9, 7] 	Entropie : 1.9730014063936123
Vecteur : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 	Entropie : 0.0


On code la méthode *entropie_cond* qui à partir d'une liste de vecteurs, calcule l'homogéinité de la partition obtenue. Il s'agit de la moyenne pondérée des entropies des sous-ensembles obtenus.

On réalise un test avec les vecteurs créés à l'étape précédente.

In [13]:
# Test d'entropie conditionnelle
print("Ensemble de vecteurs :", vT1, vT2, vT3, sep="\n")
print("\nEntropie conditionnelle de la partition :", entropie_cond([vT1, vT2, vT3]))

Ensemble de vecteurs :
[8, 6, 4, 0, 3, 2, 3, 3, 4, 6]
[8, 10, 6, 1, 0, 10, 10, 5, 9, 7]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Entropie conditionnelle de la partition : 1.2229146468544156


Afin de charger un extrait des données de la base imdb, on dispose d'un code fourni par le sujet du TME permettant le chargement d'une zone précise du fichier. On exécute le code afin de conserver en mémoire les données et ne pas avoir à les recharger à chaque utilisation.

On réalise aussi différents affichages afin de mieux comprendre les données utilisées.

In [14]:
# Data : tableau (films, features), id2titles : dictionnaire id -> titre,
# fields : id feature -> nom
[data, id2titles, fields]= pickle.load(open("imdb_extrait.pkl", "rb"))
# La derniere colonne est le vote
datax = data[:,:32]
datay = np.array([1 if x [33] > 6.5 else -1 for x in data])

# On affiche le nom des différents champs
print("Les différents champs des données dans data sont :", fields, sep="\n")

# On affiche la taille, la première instance de la base de données récupérée et son label
print("\nLe nombre d'exemples dans la base de données est de :", len(datax))
print("\nLa attributs du premier exemple sont :", ["{0:0.2f}".format(i) for i in datax[0]], sep='\n')
print("\nLe label du premier exemple est :", datay[0])

# On affiche le premier titre
print("\nLe titre du premier film de data est :", id2titles[0])

Les différents champs des données dans data sont :
{0: 'Sci-Fi', 1: 'Crime', 2: 'Romance', 3: 'Animation', 4: 'Music', 5: 'Comedy', 6: 'War', 7: 'Horror', 8: 'Film-Noir', 9: 'Adult', 10: 'News', 11: 'Reality-TV', 12: 'Thriller', 13: 'Western', 14: 'Mystery', 15: 'Short', 16: 'Talk-Show', 17: 'Drama', 18: 'Action', 19: 'Documentary', 20: 'Musical', 21: 'History', 22: 'Family', 23: 'Adventure', 24: 'Fantasy', 25: 'Game-Show', 26: 'Sport', 27: 'Biography', 28: 'Duree', 29: 'couleur', 30: 'annee', 31: 'budget', 32: 'nb_votes', 33: 'rating'}

Le nombre d'exemples dans la base de données est de : 4587

La attributs du premier exemple sont :
['0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '108.00', '2.00', '1988.00', '12324000.00']

Le label du premier exemple est : -1

Le titre du premier film de data est : 'Crocodile

On obtient donc une base de données conservée dans la variable data. Chaque ligne de data représente un film précis et on dispose de plusieurs attributs à propos de ce dernier comme son genre, sa durée, son année de sortie ou son budget par exemple.

Le genre du film est géré par un one-hot encoding sur les 28 premiers attributs. Si pour un de ses attributs on retrouve la valeur 1, alors cela signifie que le film appartient au genre représenté par l'attribut. On peut, comme pour le premier exemple, trouver un film appartenant à plusieurs catégories.

Le vecteur datay contient le score de chacun des films et ce score est égal à 1 si la moyenne de ses notes est supérieur à 6.5, ou égal à -1 dans le cas contraire.

On s'intéresse maintenant à l'entropie du vecteur datay contenant les labels de chacun des films, puis à l'entropie conditionnelle des vecteurs des 28 premiers attributs de chacun des exemples de la base de données.

In [15]:
# On affiche l'entropie de datay
entY = entropie(datay)
print("L'entropie de Y est :", entY, '\n')

# Entropie conditionnelle pour chaque attribut binaire (attributs 0 à 27)
ent_cond = np.zeros(28)

for i in range(28) :
    # On cherche quels sont les exemples dont l'attribut i est égal à 0 ou 1
    ind0 = np.where(np.array([x[i] for x in datax]) == 0)
    ind1 = np.where(np.array([x[i] for x in datax]) == 1)

    # On récupère les différents labels
    lab0 = [datay[j] for j in ind0][0]
    lab1 = [datay[j] for j in ind1][0]

    # On sauvegarde l'entropie conditionnelle sur l'attribut i
    ent_cond[i] = entropie_cond([lab0, lab1])

# On affiche les différentes entropies conditionnelles calculées
print("Les entropies conditionnelles en fonction de chaque attribut (0 à 27) sont :", ent_cond, sep='\n')


L'entropie de Y est : 0.6840731540145955 

Les entropies conditionnelles en fonction de chaque attribut (0 à 27) sont :
[0.67994028 0.68360929 0.6840712  0.68252302 0.68406157 0.67298199
 0.67776557 0.66834313 0.68159391 0.68407315 0.68407315 0.68407315
 0.67613533 0.68266372 0.68398548 0.6837022  0.68407315 0.64202544
 0.67267917 0.68320271 0.68236801 0.67462006 0.68276896 0.68388467
 0.68249102 0.68407315 0.68407204 0.66945832]


On comprend alors que la différence entre l'entropie de Y et l'entropie de Y conditionnellement à un des attributs i est le gain d'information obtenu grâce à l'utilisation de l'attribut i. On cherche donc à maximiser ce gain d'information en réalisant une comparaison entre l'entropie de Y et les entropies conditionnelles.

In [16]:
# On crée la liste contenant la différence d'entropie entre Y et Y conditionnellement à l'attribut i
difEnt = (entY - ent_cond).tolist()
indAtt= difEnt.index(max(difEnt))

# On affiche les résultats obtenus
print("Le gain d'information pour chacun des attributs i est :", difEnt, sep='\n')
print("\nLe meilleur attribut pour réaliser la séparation est", fields[indAtt], "( en position", indAtt, ").")

Le gain d'information pour chacun des attributs i est :
[0.00413287340530788, 0.00046386323157165954, 1.9551364983660946e-06, 0.0015501296269213594, 1.1586434889521158e-05, 0.011091159441398268, 0.006307582810820889, 0.015730023292110373, 0.002479248608378226, 0.0, 0.0, 0.0, 0.007937823466661387, 0.0014094297134645828, 8.767079884108853e-05, 0.0003709584397669774, 0.0, 0.04204771856211831, 0.011393982540818137, 0.0008704410364122062, 0.0017051473508644666, 0.009453090064388325, 0.001304193427266287, 0.00018848299466323049, 0.0015821370532405066, 0.0, 1.1176414111968569e-06, 0.014614838257637763]

Le meilleur attribut pour réaliser la séparation est Drama ( en position 17 ).


### **Quelques expériences préliminaires**



### **Sur et sous apprentissage**

### **Validation croisée et sélection de modèle**